In [12]:
import pandas as pd
from selenium import webdriver
from concurrent.futures import ThreadPoolExecutor # phân nhiều luồng chạy 
import time
import time
import random
from bs4 import BeautifulSoup

In [13]:
df_player = pd.read_csv('../data/Stats_csv/players.csv')

In [14]:
import random
from selenium.webdriver.chrome.options import Options
def init_driver():
    options = Options()
    options.add_argument('--headless')
    #options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    return webdriver.Chrome(options=options)
# Hàm xử lý từng URL

def get_HTML_Info(args):
    url, file_name = args
    try:
        driver = init_driver()
        driver.get(url)
        print(f"Collecting data from {url}")
        time.sleep(random.uniform(5, 7))  # Chờ ngẫu nhiên từ 5 đến 7 giây
        html_content = driver.page_source
        
        # Loại bỏ các thẻ <script> và các nội dung không mong muốn
        soup = BeautifulSoup(html_content, 'html.parser')
        for script in soup(["script", "style"]):  # Tìm và loại bỏ các thẻ <script> và <style>
            script.decompose()
        cleaned_html = soup.prettify()  # Định dạng lại HTML

        print(f"Saving cleaned data to {file_name}")
        with open(file_name, 'w', encoding='utf8') as file:
            file.write(cleaned_html)
        # PlayerInfo = getPlayerDetails(cleaned_html)
        # try:
        #     if (PlayerInfo['Position'] == 'Defender'): getDefenderInfo(html_content,PlayerInfo,file_name)
        #     elif (PlayerInfo['Position'] == 'Forward'): getForwardInfo(html_content,PlayerInfo,file_name)
        #     elif (PlayerInfo['Position'] == 'Goalkeeper'): getGoalkeeperInfo(html_content,PlayerInfo,file_name)
        #     elif (PlayerInfo['Position'] == 'Midfielder'): getMidfielderInfo(html_content,PlayerInfo,file_name)
        # except Exception as e:
        #         print(f"Skipping incomplete stat detail.{e} url: {url}")
        # finally:
        #     print(f"{file_name} saved!")
    except Exception as e:
        print(f"Error processing {url}: {e}")
    finally:
        driver.quit()
        print(f"Done processing {url}")


In [15]:
def getLink(link):
    return link.split('?')[0]
df_player['link_profile_stats_all_section'] = df_player['link_profile_stats_2425'].apply(getLink)
df_player['link_profile_stats_2324'] = df_player['link_profile_stats_all_section'] + '?co=1&se=578'
df_player['link_profile_stats_2223'] = df_player['link_profile_stats_all_section'] + '?co=1&se=489'
df_player['link_profile_stats_2122'] = df_player['link_profile_stats_all_section'] + '?co=1&se=418'
df_player['link_profile_stats_2021'] = df_player['link_profile_stats_all_section'] + '?co=1&se=363'


In [16]:
def createUrlFile(section):
    lst_name = df_player['name'].values
    urlFiles =[]
    for name in lst_name:
        urlFiles.append(f'../data/players/{name} - stats{section}.html')
    return urlFiles
def getUrls(section):
    urls = df_player[f'link_profile_stats_{section}'].values
    return urls
def file_out(section):
    file_out = df_player['position'] +section+'.csv'
    return file_out

In [18]:
df_player.loc[df_player['name'] == 'Harry Maguire']

,name,position,country,link_profile_stats_2425,link_profile_stats_all_section,link_profile_stats_2324,link_profile_stats_2223,link_profile_stats_2122,link_profile_stats_2021
566,Harry Maguire,Defender,England,https://www.premierleague.com/players/9566/Har...,https://www.premierleague.com/players/9566/Har...,https://www.premierleague.com/players/9566/Har...,https://www.premierleague.com/players/9566/Har...,https://www.premierleague.com/players/9566/Har...,https://www.premierleague.com/players/9566/Har...


In [54]:
def crawlByName(name,section):
    link = df_player.loc[df_player['name'] == name, ['link_profile_stats_2425']].values
    args = [link[0][0],f'../data/players/{name} - stats{section}.html']
    get_HTML_Info(args)

In [42]:
crawlByName('Harry Maguire','2425')

Saving cleaned data to ../data/players/Harry Maguire - stats2425.html
Done processing https://www.premierleague.com/players/9566/Harry-Maguire/stats?co=1&se=719


In [35]:
# lấy dữ liệu mùa 2023-2024
def getDataBySection(section:str):
    urls = getUrls(section)
    urlFiles = createUrlFile(section)
    # Kết URL và file thành danh sách tham số
    args_list = []
    for i in range(len(urls)):
        args_list.append([urls[i], urlFiles[i]])
    # Xử lý các URL song song với 5 luồng
    with ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(get_HTML_Info, args_list)

In [36]:
getDataBySection('2425')


Saving cleaned data to ../data/players/Zach Abbott - stats2425.html
Saving cleaned data to ../data/players/Adam Armstrong - stats2425.html
Saving cleaned data to ../data/players/Josh Acheampong - stats2425.html
Saving cleaned data to ../data/players/Max Aarons - stats2425.html
Saving cleaned data to ../data/players/Tyler Adams - stats2425.html
Done processing https://www.premierleague.com/players/125075/Josh-Acheampong/stats?co=1&se=719
Done processing https://www.premierleague.com/players/117754/Zach-Abbott/stats?co=1&se=719
Done processing https://www.premierleague.com/players/19970/Max-Aarons/stats?co=1&se=719
Done processing https://www.premierleague.com/players/42894/Tyler-Adams/stats?co=1&se=719
Done processing https://www.premierleague.com/players/9199/Adam-Armstrong/stats?co=1&se=719
Saving cleaned data to ../data/players/Simon Adingra - stats2425.html
Saving cleaned data to ../data/players/Valintino Adedokun - stats2425.html
Saving cleaned data to ../data/players/Tayo Adaramo

In [39]:
args = []
args.append(['https://www.premierleague.com/players/4664/Hugo-Lloris/stats?co=1&se=719', '../data/players/Hugo-Lloris - stats2425.html'])
args.append(['https://www.premierleague.com/players/19851/Giovani-Lo-Celso/stats?co=1&se=719', '../data/players/Giovani-Lo-Celso - stats2425.html'])
args.append(['https://www.premierleague.com/players/9566/Harry-Maguire/stats?co=1&se=719', '../data/players/Harry-Maguire - stats2425.html'])
args.append(['https://www.premierleague.com/players/23946/Paris-Maghoma/stats?co=1&se=719' , '../data/players/Paris-Maghoma - stats2425.html'])
args.append(['https://www.premierleague.com/players/12128/Mahmoud-Dahoud/stats?co=1&se=719' , '../data/players/Mahmoud-Dahoud - stats2425.html'])

for i in args:
    get_HTML_Info(i)

Saving cleaned data to ../data/players/Hugo-Lloris - stats2425.html
Done processing https://www.premierleague.com/players/4664/Hugo-Lloris/stats?co=1&se=719
Saving cleaned data to ../data/players/Giovani-Lo-Celso - stats2425.html
Done processing https://www.premierleague.com/players/19851/Giovani-Lo-Celso/stats?co=1&se=719
Saving cleaned data to ../data/players/Harry-Maguire - stats2425.html
Done processing https://www.premierleague.com/players/9566/Harry-Maguire/stats?co=1&se=719
Saving cleaned data to ../data/players/Paris-Maghoma - stats2425.html
Done processing https://www.premierleague.com/players/23946/Paris-Maghoma/stats?co=1&se=719
Saving cleaned data to ../data/players/Mahmoud-Dahoud - stats2425.html
Done processing https://www.premierleague.com/players/12128/Mahmoud-Dahoud/stats?co=1&se=719


In [9]:
df_player.loc[df_player['position']=='Defender', 'name'].values

array(['Max Aarons', 'Zach Abbott', 'Josh Acheampong', 'Tosin Adarabioyo',
       'Tayo Adaramola', 'Valintino Adedokun', 'Nayef Aguerd', 'Ola Aina',
       'Rayan Aït-Nouri', 'Kristoffer Ajer', 'Manuel Akanji',
       'Nathan Aké', 'Trent Alexander-Arnold', 'Sonny Aljofree',
       'Max Alleyne', 'Harry Amass', 'Joachim Andersen',
       'Aaron Anselmino', 'Julián Araujo', 'Benjamin Arthur',
       'Harrison Ashby', 'Noël Atom', 'Sèrge Aurier', 'Benoît Badiashile',
       'Elkan Baggott', 'Eric Bailly', 'Dominic Ball', 'Fodé Ballo-Touré',
       'Valentín Barco', 'Calvin Bassey', 'Mads Bech', 'Owen Beck',
       'Jan Bednarek', 'Armel Bella-Kotchap', 'Ben Nelson',
       'Rhys Bennett', 'Giulian Biancone', 'Bendegúz Bolla',
       'Willy-Arnaud Boly', 'Sven Botman', 'Conor Bradley',
       'Zak Bradshaw', 'Kaden Braithwaite', 'Jarrad Branthwaite',
       'James Bree', 'Josh Brooking', 'Hugo Bueno', 'Cameron Burgess',
       'Dan Burn', 'Luke Butterfield', 'Riccardo Calafiori',
       

In [55]:
crawlByName('Tosin Adarabioyo','2425')
    

Saving cleaned data to ../data/players/Tosin Adarabioyo - stats2425.html
Done processing https://www.premierleague.com/players/13549/Tosin-Adarabioyo/stats?co=1&se=719


In [57]:
crawlByName('Harry Maguire','2425')

Saving cleaned data to ../data/players/Harry Maguire - stats2425.html
Done processing https://www.premierleague.com/players/9566/Harry-Maguire/stats?co=1&se=719


In [77]:
crawlByName('Oscar Bobb','2425')

Saving cleaned data to ../data/players/Oscar Bobb - stats2425.html
Done processing https://www.premierleague.com/players/66657/Oscar-Bobb/stats?co=1&se=719
